In [ ]:
template = {
    'user_id': int,
    'name': {
        'first': str,
        'last': str
    },
    'bio': {
        'dob': {
            'year': int,
            'month': int,
            'day': int
        },
        'birthplace': {
            'country': str,
            'city': str
        }
    }
}

john = {
    'user_id': 100,
    'name': {
        'first': 'John',
        'last': 'Cleese'
    },
    'bio': {
        'dob': {
            'year': 1939,
            'month': 11,
            'day': 27
        },
        'birthplace': {
            'country': 'United Kingdom',
            'city': 'Weston-super-Mare'
        },
    }
}


def app_error(error_message):
    return f'False, {error_message}'


def is_same_type(data, original_type):
    try:
        return isinstance(data, original_type)
    except TypeError:
        return isinstance(data, dict) and isinstance(original_type, dict)


def find_mismatched_keys(keys_data, keys_template):
    mismatched_keys = list((keys_data - keys_template) or (keys_template - keys_data))
    try:
        return mismatched_keys[0]
    except IndexError:
        return None


def render_error_trace(key_default, key_errors):
    preceed_key = key_default if key_default else ''
    is_connected = '.' if preceed_key else ''
    return f'{preceed_key}{is_connected}{key_errors}'


def validate(data, template):
    state = True
    error = ''
    def inner(data, template, key_default):
        nonlocal error
        error_trace = ''

        mismatched_key = find_mismatched_keys(data.keys(), template.keys())
        if mismatched_key:
            error = 'mismatched keys'
            error_trace = render_error_trace(key_default, mismatched_key)
        else:
            for key in data:
                if not is_same_type(data[key], template[key]):
                    error = 'bad types'
                    error_trace = render_error_trace(key_default, key)

        if not error_trace:
            for key in data:
                if isinstance(data[key], dict):
                    inner_error_trace = inner(data[key], template[key], key)
                    if inner_error_trace:
                        error_trace = render_error_trace(key_default, inner_error_trace)
        return error_trace

    error_message = inner(data, template, None)
    return True if not error else app_error(f'{error}: {error_message}')


print(validate(john, template))
